In [1]:
import os
import cv2
import numpy as np
import json
import math
import pandas as pd
from sklearn import preprocessing
from itertools import cycle
from scipy import stats
from scipy.stats import mstats
from scipy.stats.mstats import winsorize
from sklearn.cluster import MiniBatchKMeans, KMeans
import phenograph
import seaborn as sns
from BorutaShap import BorutaShap, load_data

In [2]:
from Libs.Dataset import Dataset

In [482]:
def read_series(image_path):
    """Read all images from path and save it as array
    param path: directory with files, str
    return img_series: array with images, array
    return img_names: list of file names, list
    """
    all_img = []
    img_names = []
    for img_name in sorted(os.listdir(image_path), key=lambda x: int(x.split('.')[0])):
        img = cv2.imread(os.path.join(image_path, img_name), cv2.IMREAD_UNCHANGED)
        all_img.append(img)
        img_names.append(img_name)
    img_series = np.array(all_img)
    return img_names, img_series

 def calculate_properties(contour):
        """
        Calculate properties of contour
        param contour: array with contour coordinates, np.array
        return perimeter: len of the contour, float
        return radius_nuclei: radius of the contour, float
        return size: square of contour, float
        return equi_radius: radius based on square, float
        """
        perimeter = cv2.arcLength(contour, True)
        radius_nuclei = perimeter / (2 * math.pi)
        size = math.fabs(cv2.contourArea(contour))
        equi_radius = np.sqrt(size / math.pi)

        return perimeter, radius_nuclei, size, equi_radius

In [436]:
file_name = 'Fucci'
image_path = 'Dataset/'
channels = ['0', '1', '2']
mask_channel = '2'
result_path = 'Datasets/'
split_file = True

In [141]:
dataset = Dataset(file_name=file_name, image_path=image_path, channels=channels, 
                  mask_channel=mask_channel, result_path=result_path)

In [ ]:
counter = 0
for file in os.listdir(image_path):
    img_series = cv2.imreadmulti(os.path.join(image_path, file), flags=cv2.IMREAD_UNCHANGED)
    img_series = np.array(img_series[1])
    channels = cycle(channels)
    for ind, img in enumerate(img_series):
        if not cv2.imwrite(f'{os.path.join(result_path, next(channels))}/{str(counter)}.tif', img):
            raise Exception('Cannot save image')
    counter += 1
    print(counter)

In [ ]:
if split_file:
    dataset.split_image_series(image_path, file_name, result_path, channels)

In [ ]:
#channels_renaming
renamed_result_path = 'Datasets/renamed/'
for c in channels:
    path = os.path.join(result_path, c)
    print(path)
    os.makedirs(f'{renamed_result_path}{c}', exist_ok=True)
    counter = 0
    for ind in sorted(os.listdir(path), key=lambda x: int(x.split('.')[0])):
        img = cv2.imread(os.path.join(path, ind), cv2.IMREAD_UNCHANGED)
        img = cv2.convertScaleAbs(img)
        norm_img = np.zeros(img.shape)
        img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)
        cv2.imwrite(f'{renamed_result_path}{c}/{counter}.tif', img)
        counter +=1

In [33]:
masks_dir = 'Masks/Masks/2'
contours_dir = 'Masks/Contours/2'

In [24]:
conturs_path = 'Masks/Contours/2'
counter = 0
for ind in sorted(os.listdir(conturs_path), key=lambda x: int(x.split('.')[0])):
    os.rename(f'{conturs_path}/{ind}', f'{conturs_path}/{counter}.json')
    counter +=1

# Calculate base metrics

In [498]:
nuclei_dir = 'Nuclei/'

In [502]:
mask_path = 'Masks/Masks/2/'

In [529]:
nuclei_summary = []
row = []
for c in channels:
    save_nuclei_path = os.path.join(nuclei_dir, c)
    os.makedirs(save_nuclei_path, exist_ok=True)
    names, img_series = read_series(image_path=os.path.join(result_path, c))
    for ind, img in zip(names, img_series):
        ind = ind.split('.')[0]
        save_nuclei_path = os.path.join(nuclei_dir, c, str(ind))
        os.makedirs(save_nuclei_path, exist_ok=True)
#         mask = cv2.imread(f'{mask_path}{ind}.tif', cv2.IMREAD_UNCHANGED)[:,:,0]
#         new_mask = mask.copy()
#         new_mask[new_mask == 0] = 0
#         new_mask[new_mask > 1] = 255
#         contours, hierarchy = cv2.findContours(new_mask.astype('uint8'), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        contour_dir = f'{contours_dir}/{ind}.json'
        with open(contour_dir) as file:
            contours = json.load(file)
            contours = np.array(contours['contours'])
        contours = [np.array(x) for x in contours]
        for z in range(0,len(contours)):
            perimeter, radius_nucl, size, equi_radius = calculate_properties(contours[z])
            nucl_ind = str(ind)+'_'+str(z)
            x, y, w, h = cv2.boundingRect(contours[z])
            if ((h/w)>2) or ((w/h)>2):
                continue
            mask2 = np.zeros(img.shape, np.uint8)
            cv2.fillPoly(mask2, pts =[contours[z]], color=(255))
            temp2 = cv2.bitwise_and(img, img, mask=mask2)
            roi = temp2[y:y + h, x:x + w]
            mean_pixel = roi.sum()/len(np.where(roi > 0)[1])
            max_pixel = roi.max()
            row = [file_name, ind, z, nucl_ind,
            perimeter, size, equi_radius, radius_nucl, mean_pixel, max_pixel, c]
            nuclei_summary.append(row)
            cv2.imwrite(f'{save_nuclei_path}/{nucl_ind}.tif', roi)

In [532]:
df_path = 'Dataframes/'
nuclei_df = pd.DataFrame(nuclei_summary, columns = ['series', 'image_id', 'index', 'nucl_ind',
                                         'perimeter', 'size', 'equi_radius', 'radius', 'mean_pixel',
                                         'max_pixel', 'channel'])

nuclei_metric = nuclei_df.drop(columns=['mean_pixel', 'max_pixel', 'channel'])
nuclei_metric = nuclei_metric.drop_duplicates().set_index('nucl_ind').sort_index()
shorted = nuclei_df[['nucl_ind','max_pixel','channel']].fillna(0).drop_duplicates()
shorted2 = shorted.pivot(index='nucl_ind', columns='channel',values='max_pixel')
shorted2 = shorted2.sort_index()
nuclei_metric[['channel_0', 'channel_1', 'channel_2']] = shorted2[['0', '1', '2']]
nuclei_metric = nuclei_metric.fillna(0)
nuclei_metric.to_csv(f'{df_path}/max_metrics.csv')

nuclei_metric = nuclei_df.drop(columns=['mean_pixel', 'max_pixel', 'channel'])
nuclei_metric = nuclei_metric.drop_duplicates().set_index('nucl_ind').sort_index()
shorted = nuclei_df[['nucl_ind','mean_pixel','channel']].fillna(0).drop_duplicates()
shorted2 = shorted.pivot(index='nucl_ind', columns='channel',values='mean_pixel')
shorted2 = shorted2.sort_index()
nuclei_metric[['channel_0', 'channel_1', 'channel_2']] = shorted2[['0', '1', '2']]
nuclei_metric = nuclei_metric.fillna(0)
nuclei_metric.to_csv(f'{df_path}/mean_metrics.csv')

In [222]:
for img_name in os.listdir('Nuclei/2'):
    for nucl_name in os.listdir(f'Nuclei/2/{img_name}'):
        img = cv2.imread(f'Nuclei/2/{img_name}/{nucl_name}', cv2.IMREAD_UNCHANGED)
        cv2.imwrite(f'Nuclei/2_all/{nucl_name}', img)

# Texture

In [534]:
import Libs.classification as classification
from Libs.features_extraction import HaralickFeatures, TAS, \
ZernikeMoments, extract, Preprocess, CenterMass, ChromatinFeatures

In [537]:
features = {'features': [HaralickFeatures(), TAS(), ZernikeMoments(None), CenterMass(), ChromatinFeatures()],
            'feat_num' : [13, 54, 25, 2, 4],
            'objects_1' : 2230,
            'object_1_class': 'Fucci'}

In [ ]:
df = classification.dataCollection('Nuclei/2_all', features, preproc_commands = [], normalize=True, enhance=False) 

In [541]:
df.to_csv('Dataframes/textures.csv', index=False)

# Graph embedding

In [22]:
from skshape.image.segmentation import segment_phase_field
from skshape.geometry import triangulation_from_labels
from skshape.image.enhancement import weighted_smoothing

from karateclub import GeoScattering
import networkx as nx

In [27]:
def calculate_triangulation(img, mask):
    """
    Calculate triangulation in the mask and filter it to keep triangles only in the cell:
    param mask: grayscale mask, numpy array
    return curves: array with curves, numpy array
    return vertices: array with all coordinates, numpy array
    return new_triangles: array with filtered triangles, numpy array
    """
    curves, vertices, triangles = triangulation_from_labels(mask, smooth_boundaries=False, coarsen_boundaries=False)
    filter_list = np.empty(vertices.shape[0], dtype=np.float32)
    scale_x = img.shape[:2][0]
    scale_y = img.shape[:2][1]
    init_mask = (img > 0.2)
    contours_cell, hierarchy = cv2.findContours(init_mask.astype('uint8'),
                                                cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for i in range(vertices.shape[0]):
        x = scale_x * vertices[i][1]
        y = scale_y * vertices[i][0]
        coordinates = (x, y)
        filter_list[i] = cv2.pointPolygonTest(contours_cell[0], coordinates, False)
    new_triangles = []
    for i in triangles:
        counter = 0
        for x in i:
            if filter_list[x] > 0:
                continue
            else:
                counter += 1
        if counter == 0:
            new_triangles.append(i)
    new_triangles = np.array(new_triangles)

    return new_triangles

def generate_triangle_df(img_path, mask_path):

    nuclei_df2 = {}
    for img_name in os.listdir(mask_path):
        print(os.path.join(mask_path, img_name))
        mask = cv2.imread(os.path.join(mask_path, img_name), cv2.IMREAD_UNCHANGED)[:, :, 0]
        img = cv2.imread(os.path.join(img_path, img_name), cv2.IMREAD_UNCHANGED)
        new_triangles = calculate_triangulation(img, mask)
        nuclei_df2[img_name.split('.')[0]] = new_triangles

    reindexed_df2 = {}
    for key in nuclei_df2:
        G = nx.Graph()
        for path in nuclei_df2[key]:
            nx.add_path(G, path)
        mapping = {}
        counter = 0
        for n in np.unique(nuclei_df2[key]):
            mapping[n] = counter
            counter += 1
        H = nx.relabel_nodes(G, mapping, copy=True)
        if len(H) < 2:
            continue
        if not nx.is_connected(H):
            continue
        model = GeoScattering(3)
        model.fit([H])
        X = model.get_embedding()
        reindexed_df2[key] = X

    new_df2 = pd.DataFrame()
    for x in reindexed_df2:
        series = reindexed_df2[x]
        seq = pd.Series(series[0], name=x)
        new_df2 = new_df2.append(seq)

    return new_df2

In [ ]:
triangulation_df = generate_triangle_df('Nuclei/2_all', 'Masks/Nuclei_chromo')

In [30]:
triangulation_df.to_csv('Dataframes/triangulations.csv')

# Classifier embedding

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy
from PIL import Image

In [212]:
image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=224),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
}

In [15]:
class CustomResnet(nn.Module):
        def __init__(self, resnet50):
            super(CustomResnet, self).__init__()
            self.resnet50 = resnet50
            self.resnet50.fc = nn.Linear(2048, 256)
            self.internalfeature = nn.Sequential(nn.Linear(256, 32))
            self.finalfeature = nn.Sequential(nn.Linear(32, 4))
            
        def forward(self, x):
            x = F.relu(self.resnet50(x))
            x = F.relu(self.internalfeature(x))
            x = self.finalfeature(x)
            return x

In [205]:
def predict(model, test_image_name):
    transform = image_transforms['test']
    test_image = Image.open(test_image_name)
    test_image = test_image.resize((224, 224), Image.ANTIALIAS)
    test_image = test_image.convert('RGB')
    test_image_tensor = transform(test_image)
    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).to(device)
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    with torch.no_grad():
        model.eval()
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(1, dim=1)
        return topclass

In [17]:
device = 'cpu'

In [18]:
model = CustomResnet(models.resnet50(pretrained=False))
# model = models.resnet50(pretrained=True)
model.load_state_dict(torch.load('Models/classifier_custom_resnet50_4_classes.pth'))
model = model.to(device)

In [ ]:
img_dir = 'Nuclei/2_all/'
summary_dr = []
for i in os.listdir(img_dir):
    print(i)
    image_name = f'{img_dir}/{i}'
    image = Image.open(image_name)
    transform = image_transforms['test']
    feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
    test_image = image.convert('RGB')
    test_image_tensor = transform(test_image)
    test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    output = feature_extractor(test_image_tensor)
    output = output.detach().cpu().numpy()
    clas = predict(model, image_name)
    clas = clas.detach().cpu().numpy()[0][0]
    cell_name = i.split('.')[0]
    output = np.append(output, clas)
    output = np.append(output, cell_name)
    summary_dr.append(output)
summary_dr = pd.DataFrame(summary_dr)    

In [215]:
summary_dr.to_csv('Dataframes/classifier_custom_32f.csv', index=False)

# Nuclei filtration 

In [74]:
max_metrics = pd.read_csv('Dataframes/max_metrics.csv').set_index('nucl_ind')
mean_metrics = pd.read_csv('Dataframes/mean_metrics.csv').set_index('nucl_ind')

In [6]:
filter_df = pd.read_csv('Dataframes/qc_brightness.csv').set_index('cell_name')
filter_df = filter_df.loc[filter_df['type'] == 'normal']

In [ ]:
textures = pd.read_csv('Dataframes/textures.csv').set_index('cell_name')
textures = textures.loc[filter_df.index]
textures = textures.dropna()
standard_scaler = preprocessing.StandardScaler()
textures_norm = textures.copy(deep=True)
for c in textures.columns:
    x = textures[c].values.reshape(-1, 1)
    x_scaled = standard_scaler.fit_transform(x)
    textures_norm[c] = x_scaled
actual_metrics = mean_metrics.loc[textures_norm.index]
z_scores = stats.zscore(actual_metrics[['channel_0', 'channel_1']])
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
textures_norm_filt = textures_norm[filtered_entries]
print(textures.shape)
textures.head(3)

In [ ]:
mesh_embedding_df = pd.read_csv('Dataframes/triangulations.csv').set_index('Unnamed: 0')
mesh_embedding_df = mesh_embedding_df.loc[filter_df.index]
mesh_embedding_df = mesh_embedding_df.dropna()
standard_scaler = preprocessing.StandardScaler()
mesh_norm = mesh_embedding_df.copy(deep=True)
for c in mesh_norm.columns:
    x = mesh_norm[c].values.reshape(-1, 1)
    x_scaled = standard_scaler.fit_transform(x)
    mesh_norm[c] = x_scaled
actual_metrics = mean_metrics.loc[mesh_norm.index]
z_scores = stats.zscore(actual_metrics[['channel_0', 'channel_1']])
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
mesh_norm_filt = mesh_norm[filtered_entries]
print(mesh_norm_filt.shape)
mesh_norm_filt.head(3)

In [64]:
classifier_df = pd.read_csv('Dataframes/classifier_custom_32f.csv').set_index('33', drop=True).drop(['32'], axis=1)
classifier_df = classifier_df.loc[filter_df.index]
standard_scaler = preprocessing.StandardScaler()
classifier_norm = classifier_df.copy(deep=True)
for c in classifier_df.columns:
    x = classifier_df[c].values.reshape(-1, 1)
    x_scaled = standard_scaler.fit_transform(x)
    classifier_norm[c] = x_scaled
actual_metrics = mean_metrics.loc[classifier_norm.index]
z_scores = stats.zscore(actual_metrics[['channel_0', 'channel_1']])
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
classifier_norm_filt = classifier_norm[filtered_entries]
classifier_norm_filt.shape
classifier_norm_filt.head(3)

# Clusterization

In [ ]:
#Textures clusterization
textures_clust = textures_norm_filt.copy(deep=True)
communities, graph, Q = phenograph.cluster(textures_clust, primary_metric='cosine', n_jobs=10, k=350, min_cluster_size=5, clustering_algo='leiden')
communities = pd.Series(communities, name='phenograph_cluster', index=textures_clust.index)
textures_clustered = pd.concat([textures_clust, communities], axis=1)
print(communities.unique())

In [ ]:
#Textures_clusters
actual_metrics = mean_metrics.loc[textures_clustered.index]
textures_clustered_vis = textures_clustered.copy(deep=True)
textures_clustered_vis[['channel_0', 'channel_1']] = actual_metrics[['channel_0', 'channel_1']]
actual_metrics = mean_metrics.loc[textures_clustered_vis.index]
z_scores = stats.zscore(actual_metrics[['channel_0', 'channel_1']])
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
textures_clustered_vis = textures_clustered_vis[filtered_entries]
stand_scaler = preprocessing.MinMaxScaler()
x = textures_clustered_vis['channel_0'].values.reshape(-1, 1)
x_scaled = stand_scaler.fit_transform(x)
textures_clustered_vis['channel_0'] = x_scaled
x = textures_clustered_vis['channel_1'].values.reshape(-1, 1)
x_scaled = stand_scaler.fit_transform(x)
textures_clustered_vis['channel_1'] = x_scaled

g = sns.FacetGrid(textures_clustered_vis, col="phenograph_cluster", hue="phenograph_cluster", col_wrap=5)
# g = sns.FacetGrid(textures_clustered_vis, col="k_cluster", hue="k_cluster", col_wrap=5)
g.map(sns.scatterplot, "channel_0", "channel_1", s=20)

In [ ]:
#Mesh clusterization
mesh_embedding_df_clust = mesh_norm_filt.copy(deep=True)
communities, graph, Q = phenograph.cluster(mesh_embedding_df_clust, primary_metric='cosine', n_jobs=10, k=350, min_cluster_size=5)
communities = pd.Series(communities, name='phenograph_cluster', index=mesh_embedding_df_clust.index)
mesh_clustered = pd.concat([mesh_embedding_df_clust, communities], axis=1)
print(communities.unique())

In [ ]:
#Classifier clusterization
classifier_df = classifier_norm_filt.copy(deep=True)
communities, graph, Q = phenograph.cluster(classifier_df, primary_metric='cosine', n_jobs=-1, k=350, min_cluster_size=5, clustering_algo='louvain')
communities = pd.Series(communities, name='phenograph_cluster', index=classifier_df.index)
comb_clustered = pd.concat([classifier_df, communities], axis=1)
print(communities.unique())

In [ ]:
#Classifier_clusters
actual_metrics = mean_metrics.loc[classifier_clustered.index]
classifier_clustered_vis2 = comb_clustered
classifier_clustered_vis2[['channel_0','channel_1']] = actual_metrics[['channel_0', 'channel_1']]
z_scores = stats.zscore(classifier_clustered_vis2[['channel_0', 'channel_1']])
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
classifier_clustered_vis2_filt = classifier_clustered_vis2[filtered_entries]

min_max_scaler = preprocessing.MinMaxScaler()
x = classifier_clustered_vis2['channel_1'].values.reshape(-1, 1)
x_scaled = min_max_scaler.fit_transform(x)
classifier_clustered_vis2['channel_1'] = x_scaled
x = classifier_clustered_vis2['channel_0'].values.reshape(-1, 1)
x_scaled = min_max_scaler.fit_transform(x)
classifier_clustered_vis2['channel_0'] = x_scaled

g = sns.FacetGrid(classifier_clustered_vis2, col="phenograph_cluster", hue="phenograph_cluster", col_wrap=5)
# g = sns.FacetGrid(classifier_clustered_vis2, col="k_cluster", hue="k_cluster", col_wrap=5)
g.map(sns.scatterplot, "channel_0", "channel_1", size=1)

# Visualisation

In [19]:
from umap import UMAP

In [ ]:
#Textures
textures_matr = textures_clustered.drop(['phenograph_cluster'], axis=1)
projection_model = UMAP(n_components=2, n_neighbors=350,  metric='cosine',
                 random_state=42, transform_seed=42)
projection = pd.DataFrame(projection_model.fit_transform(textures_matr),
                          columns=['v0', 'v1'])
# projection_model = MulticoreTSNE.MulticoreTSNE(n_jobs=10, perplexity=350, random_state=42,n_iter=5000)
# projection = projection_model.fit_transform(sample_data_matr)
cluster_centers = pd.concat([projection, textures_clustered['phenograph_cluster'].reset_index()], axis=1)
cluster_centers = cluster_centers.groupby('phenograph_cluster').median().reset_index()

cluster_pallete = {
    i[1]: i[0]
    for i in zip(sns.color_palette('Set2', n_colors=len(textures_clustered['phenograph_cluster'].unique())),
                 textures_clustered['phenograph_cluster'].unique())
}
ax.scatter(projection['v0'],
           projection['v1'],
           s=60,
           c=textures_clustered['phenograph_cluster'].map(cluster_pallete)
)
for _, row in cluster_centers.iterrows():
    ax.annotate(row['phenograph_cluster'], row[['v0', 'v1']], size=5)
plt.tight_layout()

In [ ]:
# #Mesh
mesh_matr = mesh_clustered.drop(['phenograph_cluster'], axis=1)
projection_model = UMAP(n_components=2, n_neighbors=50,  metric='correlation',
                 random_state=42, transform_seed=42)
projection = pd.DataFrame(projection_model.fit_transform(mesh_matr),
                          columns=['v0', 'v1'])

# projection_model = MulticoreTSNE.MulticoreTSNE(n_jobs=10, perplexity=350, random_state=42,n_iter=5000)
# projection = projection_model.fit_transform(sample_data_matr)
cluster_centers = pd.concat([projection, mesh_clustered['phenograph_cluster'].reset_index()], axis=1)
cluster_centers = cluster_centers.groupby('phenograph_cluster').median().reset_index()

cluster_pallete = {
    i[1]: i[0]
    for i in zip(sns.color_palette('Set2', n_colors=len(mesh_clustered['phenograph_cluster'].unique())),
                 mesh_clustered['phenograph_cluster'].unique())
}
ax.scatter(projection['v0'],
           projection['v1'],
           s=6,
           c=mesh_clustered['phenograph_cluster'].map(cluster_pallete)
)
for _, row in cluster_centers.iterrows():
    ax.annotate(row['phenograph_cluster'], row[['v0', 'v1']], size=5)
plt.tight_layout()

In [ ]:
# #Classifier
classifier_matr = classifier_clustered_vis.drop(['phenograph_cluster'], axis=1)
projection_model = UMAP(n_components=2, n_neighbors=350,  metric='correlation',
                 random_state=42, transform_seed=42)
projection = pd.DataFrame(projection_model.fit_transform(classifier_matr),
                          columns=['v0', 'v1'])

# projection_model = MulticoreTSNE.MulticoreTSNE(n_jobs=10, perplexity=350, random_state=42,n_iter=5000)
# projection = projection_model.fit_transform(sample_data_matr)
cluster_centers = pd.concat([projection, classifier_clustered_vis['phenograph_cluster'].reset_index()], axis=1)
cluster_centers = cluster_centers.groupby('phenograph_cluster').median().reset_index()

cluster_pallete = {
    i[1]: i[0]
    for i in zip(sns.color_palette('Set2', n_colors=len(classifier_clustered_vis['phenograph_cluster'].unique())),
                 classifier_clustered_vis['phenograph_cluster'].unique())
}
ax.scatter(projection['v0'],
           projection['v1'],
           s=2,
           c=classifier_clustered_vis['phenograph_cluster'].map(cluster_pallete)
)
for _, row in cluster_centers.iterrows():
    ax.annotate(row['phenograph_cluster'], row[['v0', 'v1']], size=5)
plt.tight_layout()

# Feature selection

In [ ]:
subsample = textures_clustered_vis.groupby('phenograph_cluster', group_keys=False).apply(lambda x: x.sample(80))

In [ ]:
df_for_select = subsample
X, y = df_for_select.drop(['phenograph_cluster'], axis=1), df_for_select['phenograph_cluster']

In [ ]:
from xgboost import XGBClassifier,XGBRegressor
model = XGBClassifier()
Feature_Selector = BorutaShap(model=model, importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=X, y=y, n_trials=30, random_state=42)

In [ ]:
Feature_Selector.TentativeRoughFix()
Feature_Selector.plot(which_features='all', 
                      X_size=8, figsize=(12,8),
                      y_scale='log')

In [130]:
metric = ['11', '0', '4', '22', '13', '14', '2', '18', '19', '9', '3', '8', '26', '6', '5', '23', '29']
metric2 = ['ZernikeMoments_#24', 'HaralickFeatures_#12', 'ZernikeMoments_#17', 'TAS_#02', 'ZernikeMoments_#23', 'TAS_#10', 'ZernikeMoments_#06', 'TAS_#01', 'HaralickFeatures_#05', 'channel_1', 'TAS_#19', 'ZernikeMoments_#22', 'HaralickFeatures_#08', 'ZernikeMoments_#13', 'TAS_#26', 'TAS_#46', 'TAS_#39', 'HaralickFeatures_#11', 'TAS_#37', 'ZernikeMoments_#04', 'TAS_#29', 'TAS_#11', 'HaralickFeatures_#13', 'ZernikeMoments_#15', 'ZernikeMoments_#14', 'ZernikeMoments_#03', 'TAS_#08', 'HaralickFeatures_#06', 'TAS_#18', 'TAS_#28', 'TAS_#30', 'HaralickFeatures_#09', 'ZernikeMoments_#12', 'ZernikeMoments_#07', 'TAS_#43', 'TAS_#20', 'ZernikeMoments_#05', 'TAS_#09']